In [1]:
import pandas as pd

from mainnet_launch.autopool_diagnostics.compute_rebalance_cost import (
    eth_client,
    AutopoolConstants,
    fetch_spot_value_of_rebalances_from_dest_to_idle,
    AUTO_LRT,
)
import plotly.express as px

df = pd.read_csv("gwei_percentiles.csv", index_col=1, parse_dates=True)

2024-11-15 15:35:16.860 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-11-15 15:35:16.868 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-11-15 15:35:16.870 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-11-15 15:35:16.872 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [2]:
df

,block_number,gwei_5th_percentile,gwei_10th_percentile,gwei_20th_percentile,gwei_30th_percentile,gwei_40th_percentile,gwei_50th_percentile,gwei_60th_percentile,gwei_70th_percentile,gwei_80th_percentile,gwei_90th_percentile,gwei_95th_percentile
block_timestamp,,,,,,,,,,,,
2024-05-15 04:06:11+00:00,19872880,3.91,3.91,3.97,4.00,4.00,4.00,4.02,4.88,5.88,6.29,12.00
2024-05-15 04:06:23+00:00,19872881,4.13,4.13,4.13,4.16,4.16,4.21,4.52,5.14,6.13,7.01,9.14
2024-05-15 04:06:35+00:00,19872882,4.44,4.44,4.44,4.50,4.51,4.91,5.16,6.00,6.41,6.54,9.00
2024-05-15 04:06:47+00:00,19872883,4.40,4.43,4.43,4.50,4.90,5.40,5.41,6.40,6.40,6.40,6.40
2024-05-15 04:06:59+00:00,19872884,4.38,4.39,4.41,4.41,4.41,4.42,5.00,5.38,6.26,6.38,7.12
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-15 01:47:11+00:00,21189905,15.29,15.30,15.36,15.75,16.28,16.43,17.28,17.28,18.32,18.68,20.29
2024-11-15 01:47:23+00:00,21189906,14.41,14.53,15.00,15.00,15.00,15.00,15.34,15.87,16.34,18.68,24.34
2024-11-15 01:47:35+00:00,21189907,14.51,14.58,14.81,15.00,15.06,15.50,16.00,16.34,16.50,16.50,18.68


In [3]:
# claim: if we submit a tx with gwei > the 5th percentile of gwei at that block it will be included
# this is not exactly correct because some these low gwei transactions can be bribing the block builder in some other way
# also a MEV searcher could be bribing the block builder to include these low gwei transactions so that they can sandwich them
# imo, its a safe assumption to be close enough

df["last_block_5th_percentile"] = df["gwei_5th_percentile"].shift(1)
100 * (df["last_block_5th_percentile"] + 1 > df["gwei_5th_percentile"]).value_counts(normalize=True)

True     90.445168
False     9.554832
Name: proportion, dtype: float64

In [4]:
calculator_gas_costs_df = pd.read_csv(
    "/home/parker/Documents/Tokemak/v2-rebalance-dashboard/mainnet_launch/working_data/chainlink_keeper_upkeeper_df.csv"
)
calculator_gas_costs_df["what_we_paid_gasPrice_in_gwei"] = calculator_gas_costs_df["gasPrice"] / 1e9
calculator_gas_costs_df[["what_we_paid_gasPrice_in_gwei", "block"]]

,what_we_paid_gasPrice_in_gwei,block
0,1.462960,20752581
1,1.507134,20753179
2,1.650092,20753236
3,1.506129,20753247
4,1.470003,20753480
...,...,...
3893,22.766418,21194744
3894,23.738541,21194746
3895,21.087497,21194879
3896,17.869943,21195045


In [5]:
df

,block_number,gwei_5th_percentile,gwei_10th_percentile,gwei_20th_percentile,gwei_30th_percentile,gwei_40th_percentile,gwei_50th_percentile,gwei_60th_percentile,gwei_70th_percentile,gwei_80th_percentile,gwei_90th_percentile,gwei_95th_percentile,last_block_5th_percentile
block_timestamp,,,,,,,,,,,,,
2024-05-15 04:06:11+00:00,19872880,3.91,3.91,3.97,4.00,4.00,4.00,4.02,4.88,5.88,6.29,12.00,NaN
2024-05-15 04:06:23+00:00,19872881,4.13,4.13,4.13,4.16,4.16,4.21,4.52,5.14,6.13,7.01,9.14,3.91
2024-05-15 04:06:35+00:00,19872882,4.44,4.44,4.44,4.50,4.51,4.91,5.16,6.00,6.41,6.54,9.00,4.13
2024-05-15 04:06:47+00:00,19872883,4.40,4.43,4.43,4.50,4.90,5.40,5.41,6.40,6.40,6.40,6.40,4.44
2024-05-15 04:06:59+00:00,19872884,4.38,4.39,4.41,4.41,4.41,4.42,5.00,5.38,6.26,6.38,7.12,4.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-15 01:47:11+00:00,21189905,15.29,15.30,15.36,15.75,16.28,16.43,17.28,17.28,18.32,18.68,20.29,15.27
2024-11-15 01:47:23+00:00,21189906,14.41,14.53,15.00,15.00,15.00,15.00,15.34,15.87,16.34,18.68,24.34,15.29
2024-11-15 01:47:35+00:00,21189907,14.51,14.58,14.81,15.00,15.06,15.50,16.00,16.34,16.50,16.50,18.68,14.41


In [8]:
our_costs_df = pd.merge(
    calculator_gas_costs_df[["what_we_paid_gasPrice_in_gwei", "block"]],
    df,
    left_on="block",
    right_on="block_number",
    how="left",
).dropna()
our_costs_df
# px.lineour_costs_df[['what_we_paid_gasPrice_in_gwei', 'gwei_5th_percentile']]

,what_we_paid_gasPrice_in_gwei,block,block_number,gwei_5th_percentile,gwei_10th_percentile,gwei_20th_percentile,gwei_30th_percentile,gwei_40th_percentile,gwei_50th_percentile,gwei_60th_percentile,gwei_70th_percentile,gwei_80th_percentile,gwei_90th_percentile,gwei_95th_percentile,last_block_5th_percentile
0,1.462960,20752581,20752581.0,0.97,0.97,0.99,1.01,1.06,1.46,1.46,1.96,2.00,2.96,3.44,0.92
1,1.507134,20753179,20753179.0,1.01,1.02,1.02,1.05,1.10,1.13,1.51,1.82,2.01,3.01,3.76,1.03
2,1.650092,20753236,20753236.0,1.10,1.10,1.11,1.18,1.60,1.63,2.00,2.10,3.10,5.20,10.00,1.02
3,1.506129,20753247,20753247.0,1.01,1.01,1.05,1.15,1.51,1.51,1.51,2.00,3.01,4.01,6.07,1.04
4,1.470003,20753480,20753480.0,0.97,0.97,1.00,1.05,1.26,1.47,1.47,2.00,2.97,3.97,3.97,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3873,30.840161,21187885,21187885.0,28.85,28.87,28.94,29.34,29.84,30.78,30.84,31.07,33.01,33.84,38.44,25.74
3874,30.999199,21187886,21187886.0,29.10,29.17,29.44,30.00,30.90,31.00,31.00,31.50,33.16,36.00,76.96,28.85
3875,25.642338,21189375,21189375.0,23.43,23.48,23.86,24.38,25.32,25.38,25.38,25.38,25.38,26.38,29.69,23.86
3876,19.827937,21189674,21189674.0,18.01,18.03,18.10,18.17,18.20,19.00,19.88,19.97,20.00,20.30,20.90,18.48


In [19]:
our_costs_df[our_costs_df["block"] == 21187886]

,what_we_paid_gasPrice_in_gwei,block,block_number,gwei_5th_percentile,gwei_10th_percentile,gwei_20th_percentile,gwei_30th_percentile,gwei_40th_percentile,gwei_50th_percentile,gwei_60th_percentile,gwei_70th_percentile,gwei_80th_percentile,gwei_90th_percentile,gwei_95th_percentile,last_block_5th_percentile
3874,30.999199,21187886,21187886.0,29.1,29.17,29.44,30.0,30.9,31.0,31.0,31.5,33.16,36.0,76.96,28.85


In [25]:
blocks_after_our_tx = df[
    (df["block_number"] < 21187886 + 300) & (df["block_number"] > 21187886)
].copy()  # about an hour
blocks_after_our_tx["gwei_5th_percentile"].describe()

count    299.000000
mean      25.920033
std        2.827571
min       20.230000
25%       23.445000
50%       25.870000
75%       28.240000
max       37.430000
Name: gwei_5th_percentile, dtype: float64

In [ ]:
hour_of_blocks_before_our_tx = df[(df["block_number"] < 21187886) & (df["block_number"] > 21187886 - 300)].copy()
hour_of_blocks_before_our_tx  # we look at the last hour

,block_number,gwei_5th_percentile,gwei_10th_percentile,gwei_20th_percentile,gwei_30th_percentile,gwei_40th_percentile,gwei_50th_percentile,gwei_60th_percentile,gwei_70th_percentile,gwei_80th_percentile,gwei_90th_percentile,gwei_95th_percentile,last_block_5th_percentile
block_timestamp,,,,,,,,,,,,,
2024-11-14 18:01:11+00:00,21187587,26.58,26.61,26.67,27.57,27.57,28.51,28.79,29.30,30.41,34.23,34.57,30.27
2024-11-14 18:01:23+00:00,21187588,29.32,29.34,29.41,30.31,31.25,31.31,31.31,31.61,32.49,33.00,36.17,26.58
2024-11-14 18:01:35+00:00,21187589,28.10,28.16,28.40,29.06,29.28,30.00,30.06,30.56,31.47,34.57,36.17,29.32
2024-11-14 18:01:47+00:00,21187590,28.20,28.28,29.18,30.07,30.12,30.18,31.00,31.00,31.51,32.25,36.17,28.10
2024-11-14 18:01:59+00:00,21187591,27.58,27.62,27.81,28.07,28.57,29.51,29.57,29.75,31.00,31.96,34.67,28.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-14 19:00:23+00:00,21187881,26.52,26.55,26.95,27.51,28.42,28.51,28.61,30.35,31.34,31.89,36.57,25.07
2024-11-14 19:00:35+00:00,21187882,27.24,27.30,27.94,28.23,29.17,29.23,29.23,29.23,30.00,32.09,32.51,26.52
2024-11-14 19:00:47+00:00,21187883,27.37,27.37,27.71,28.27,28.77,29.19,29.22,29.27,30.25,32.27,34.24,27.24


In [33]:
target_gas_price = hour_of_blocks_before_our_tx[
    "gwei_5th_percentile"
].median()  # median 5th percenile of the last hour of blocks

In [36]:
(blocks_after_our_tx["gwei_5th_percentile"] > target_gas_price).value_counts()

gwei_5th_percentile
True     185
False    114
Name: count, dtype: int64

In [9]:
our_costs_df.columns

Index(['what_we_paid_gasPrice_in_gwei', 'block', 'block_number',
       'gwei_5th_percentile', 'gwei_10th_percentile', 'gwei_20th_percentile',
       'gwei_30th_percentile', 'gwei_40th_percentile', 'gwei_50th_percentile',
       'gwei_60th_percentile', 'gwei_70th_percentile', 'gwei_80th_percentile',
       'gwei_90th_percentile', 'gwei_95th_percentile',
       'last_block_5th_percentile'],
      dtype='object')

In [ ]:
df["median_gas_price_of_prior_hour"] = df["gwei_5th_percentile"].shift(300).rolling(300).median()
df["lowest_gas_price_in_next_hour"] = df["gwei_5th_percentile"].rolling(300).min()

# if we pick the median gas price
(df["median_gas_price_of_prior_hour"] > df["lowest_gas_price_in_next_hour"]).value_counts(normalize=True)

True     0.941799
False    0.058201
Name: proportion, dtype: float64

In [ ]:
# any prediced gas price > this value is a win, get to the lowest value
df

,block_number,gwei_5th_percentile,gwei_10th_percentile,gwei_20th_percentile,gwei_30th_percentile,gwei_40th_percentile,gwei_50th_percentile,gwei_60th_percentile,gwei_70th_percentile,gwei_80th_percentile,gwei_90th_percentile,gwei_95th_percentile,last_block_5th_percentile,lowest_gas_price_in_next_hour
block_timestamp,,,,,,,,,,,,,,
2024-05-15 04:06:11+00:00,19872880,3.91,3.91,3.97,4.00,4.00,4.00,4.02,4.88,5.88,6.29,12.00,NaN,NaN
2024-05-15 04:06:23+00:00,19872881,4.13,4.13,4.13,4.16,4.16,4.21,4.52,5.14,6.13,7.01,9.14,3.91,NaN
2024-05-15 04:06:35+00:00,19872882,4.44,4.44,4.44,4.50,4.51,4.91,5.16,6.00,6.41,6.54,9.00,4.13,NaN
2024-05-15 04:06:47+00:00,19872883,4.40,4.43,4.43,4.50,4.90,5.40,5.41,6.40,6.40,6.40,6.40,4.44,NaN
2024-05-15 04:06:59+00:00,19872884,4.38,4.39,4.41,4.41,4.41,4.42,5.00,5.38,6.26,6.38,7.12,4.40,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-15 01:47:11+00:00,21189905,15.29,15.30,15.36,15.75,16.28,16.43,17.28,17.28,18.32,18.68,20.29,15.27,13.76
2024-11-15 01:47:23+00:00,21189906,14.41,14.53,15.00,15.00,15.00,15.00,15.34,15.87,16.34,18.68,24.34,15.29,13.76
2024-11-15 01:47:35+00:00,21189907,14.51,14.58,14.81,15.00,15.06,15.50,16.00,16.34,16.50,16.50,18.68,14.41,13.76


In [ ]:
# mot sure here what could be a good target?

In [50]:
px.line(our_costs_df[["what_we_paid_gasPrice_in_gwei", "gwei_50th_percentile", "last_block_5th_percentile"]])